In [1]:
import os
from pyspark import SparkContext
sc = SparkContext(appName="search", master=os.environ['MASTER'])
import pysam

In [2]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

In [4]:
#read bam file and put identicalID, tlen and read seq into sparkContext
filename = "../../../GenomeData/NA21144.chrom20.ILLUMINA.bwa.GIH.low_coverage.20130415.bam"
samfile = pysam.AlignmentFile(filename, 'rb')
tlenList = []
for ite in samfile.fetch():
    if abs(ite.tlen) > 1000:
        tlenList.append(("NA21144", ite.qname, ite.flag, ite.rname, ite.pos, ite.mapq, ite.cigar, ite.rnext, \
                         ite.pnext, ite.tlen, ite.seq, ite.qual, ite.tags))

lines = sc.parallelize(tlenList)  

#populate data into sqlContext
seqVar = lines.map(lambda p: Row(iid=p[0], tlen=p[9], seq=p[10]))
seqVarTable = sqlCtx.inferSchema(seqVar)  # pop memory exceeds error here when trying to transform all data in bam to sqlCtx
seqVarTable.registerAsTable("seqVar")


#seqVarTable.printSchema()
#print seqVarTable.collect()
#seqList = sqlCtx.sql("SELECT seq from seqVar")
